In [135]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import pipeline
from sklearn.preprocessing import RobustScaler,PowerTransformer,MinMaxScaler,StandardScaler
from sklearn.impute import SimpleImputer as Imputer

# Features Engineering and Selection

# APPLYING FEATURES

In [136]:
df = pd.read_csv("D:\data prep\dseb63_final_project_DP_dataset\dseb63_final_project_DP_dataset\dseb63_bureau.csv")
df.head()

,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,SK_ID_CURR
0,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN,254629
1,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN,254629
2,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,254629
3,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,254629
4,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,254629


In [137]:
#chuyển ngày thành năm
df['DAYS_CREDIT_ENDDATE'][df['DAYS_CREDIT_ENDDATE'] > -50*365] = np.nan
df['DAYS_ENDDATE_FACT'][df['DAYS_ENDDATE_FACT'] > -50*365] = np.nan
df['DAYS_CREDIT_UPDATE'][df['DAYS_CREDIT_UPDATE'] > -50*365] = np.nan

C:\Users\Admin\AppData\Local\Temp\ipykernel_4528\1722810815.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DAYS_CREDIT_ENDDATE'][df['DAYS_CREDIT_ENDDATE'] > -50*365] = np.nan
C:\Users\Admin\AppData\Local\Temp\ipykernel_4528\1722810815.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DAYS_ENDDATE_FACT'][df['DAYS_ENDDATE_FACT'] > -50*365] = np.nan
C:\Users\Admin\AppData\Local\Temp\ipykernel_4528\1722810815.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [138]:
#Drop cột có nhiều null, distribution chênh lệch và cột có correlation cao
new_df = df.drop(columns=['CREDIT_CURRENCY','DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE','DAYS_CREDIT_ENDDATE'])

In [139]:
new_df.head()

,SK_ID_BUREAU,CREDIT_ACTIVE,DAYS_CREDIT,CREDIT_DAY_OVERDUE,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,AMT_ANNUITY,SK_ID_CURR
0,5714462,Closed,-497,0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,NaN,254629
1,5714463,Active,-208,0,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,NaN,254629
2,5714464,Active,-203,0,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,NaN,254629
3,5714465,Active,-203,0,NaN,0,90000.0,NaN,NaN,0.0,Credit card,NaN,254629
4,5714466,Active,-629,0,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,NaN,254629


Number of past loans per customer

In [140]:
num_past_loan = new_df[['SK_ID_CURR', 'DAYS_CREDIT']].groupby('SK_ID_CURR')['DAYS_CREDIT'].count().reset_index()
num_past_loan.columns = ['SK_ID_CURR','PAST_LOAN_COUNT']
num_past_loan

,SK_ID_CURR,PAST_LOAN_COUNT
0,0,1
1,1,1
2,2,6
3,3,7
4,4,6
...,...,...
263486,307504,4
263487,307505,2
263488,307506,4
263489,307508,7


Number of types of past loans per customer

In [141]:
num_of_type = new_df[['SK_ID_CURR','CREDIT_TYPE']].groupby(['SK_ID_CURR']).nunique().reset_index()
num_of_type.columns = ['SK_ID_CURR','CREDIT_TYPE_COUNT']
num_of_type

,SK_ID_CURR,CREDIT_TYPE_COUNT
0,0,1
1,1,1
2,2,1
3,3,2
4,4,2
...,...,...
263486,307504,1
263487,307505,2
263488,307506,1
263489,307508,3


The percentage of active loans per customer

In [142]:
new_df['CREDIT_ACTIVE_BINARY'] = new_df['CREDIT_ACTIVE']

def check(status):
    if status == 'Closed':
        result = 0
    else:
        result = 1 
    return result

new_df['CREDIT_ACTIVE_BINARY'] = new_df.apply(lambda x: check(x.CREDIT_ACTIVE), axis = 1)

atv_loans = new_df.groupby(by = ['SK_ID_CURR'])['CREDIT_ACTIVE_BINARY'].mean().reset_index().rename(index=str, columns={'CREDIT_ACTIVE_BINARY': 'ACTIVE_LOANS_PERCENTAGE'})
atv_loans['ACTIVE_LOANS_PERCENTAGE'] = atv_loans['ACTIVE_LOANS_PERCENTAGE'] * 100
new_df = new_df.drop(columns=['CREDIT_ACTIVE_BINARY'])
atv_loans


,SK_ID_CURR,ACTIVE_LOANS_PERCENTAGE
0,0,100.000000
1,1,100.000000
2,2,33.333333
3,3,42.857143
4,4,50.000000
...,...,...
263486,307504,25.000000
263487,307505,100.000000
263488,307506,0.000000
263489,307508,57.142857


The Ratio of Total Debt to Total Credit for each Customer

In [143]:
data_with_sum_debt_pos_sign_all = new_df[new_df['AMT_CREDIT_SUM_DEBT']>0]

grp1 = data_with_sum_debt_pos_sign_all[['SK_ID_CURR', 'AMT_CREDIT_SUM_DEBT']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM_DEBT'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM_DEBT': 'TOTAL_CUSTOMER_DEBT'})
grp2 = new_df[['SK_ID_CURR', 'AMT_CREDIT_SUM']].groupby(by = ['SK_ID_CURR'])['AMT_CREDIT_SUM'].sum().reset_index().rename( index = str, columns = { 'AMT_CREDIT_SUM': 'TOTAL_CUSTOMER_CREDIT'})

new_df = new_df.merge(grp1, on = ['SK_ID_CURR'], how = 'left')
new_df = new_df.merge(grp2, on = ['SK_ID_CURR'], how = 'left')
del grp1, grp2


new_df['DEBT_CREDIT_RATIO'] = (new_df['TOTAL_CUSTOMER_DEBT']/new_df['TOTAL_CUSTOMER_CREDIT'])*100
new_df['DEBT_CREDIT_RATIO'] = new_df['DEBT_CREDIT_RATIO'].replace([np.inf, -np.inf], np.nan)

del new_df['TOTAL_CUSTOMER_DEBT'], new_df['TOTAL_CUSTOMER_CREDIT']

debt_credit = new_df.groupby(by = ['SK_ID_CURR'])['DEBT_CREDIT_RATIO'].mean().reset_index().rename(index=str)
debt_credit

,SK_ID_CURR,DEBT_CREDIT_RATIO
0,0,90.706731
1,1,NaN
2,2,37.908544
3,3,41.072109
4,4,44.445915
...,...,...
263486,307504,13.827518
263487,307505,NaN
263488,307506,NaN
263489,307508,4.571994


In [144]:
final_dataset = new_df.merge(num_past_loan,
                                      how='left', 
                                      on='SK_ID_CURR')
final_dataset = final_dataset.merge(num_of_type,
                                      how='left', 
                                      on='SK_ID_CURR')
final_dataset = final_dataset.merge(atv_loans,
                                      how='left', 
                                      on='SK_ID_CURR')
final_dataset = final_dataset.merge(debt_credit,
                                      how='left', 
                                      on='SK_ID_CURR')

In [145]:
final_dataset = final_dataset.drop(columns = ['DEBT_CREDIT_RATIO_y'])
final_dataset = final_dataset.rename(columns = {'DEBT_CREDIT_RATIO_x': 'DEBT_CREDIT_RATIO'})

In [146]:
final_dataset.head()

,SK_ID_BUREAU,CREDIT_ACTIVE,DAYS_CREDIT,CREDIT_DAY_OVERDUE,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,AMT_ANNUITY,SK_ID_CURR,DEBT_CREDIT_RATIO,PAST_LOAN_COUNT,CREDIT_TYPE_COUNT,ACTIVE_LOANS_PERCENTAGE
0,5714462,Closed,-497,0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,NaN,254629,4.761731,11,3,54.545455
1,5714463,Active,-208,0,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,NaN,254629,4.761731,11,3,54.545455
2,5714464,Active,-203,0,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,NaN,254629,4.761731,11,3,54.545455
3,5714465,Active,-203,0,NaN,0,90000.0,NaN,NaN,0.0,Credit card,NaN,254629,4.761731,11,3,54.545455
4,5714466,Active,-629,0,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,NaN,254629,4.761731,11,3,54.545455


# Merge With Bureau Balance

In [147]:
df2 = pd.read_csv("D:\data prep\dseb63_final_project_DP_dataset\dseb63_final_project_DP_dataset\dseb63_bureau_balance.csv")
df2.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [148]:
def extract_mean(x):
    y = x.groupby('SK_ID_BUREAU', as_index=False).mean().add_prefix('BUR_BAL_MEAN_') 
    return y

In [149]:
# apply formula to create bureau_balance dataframe grouped by SK_ID_BUREAU with mean values of all numerical columns
bureau_bal_mean = extract_mean(df2)

C:\Users\Admin\AppData\Local\Temp\ipykernel_4528\3099452620.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  y = x.groupby('SK_ID_BUREAU', as_index=False).mean().add_prefix('BUR_BAL_MEAN_')


In [150]:
bureau_bal_mean.head()

,BUR_BAL_MEAN_SK_ID_BUREAU,BUR_BAL_MEAN_MONTHS_BALANCE
0,5001709,-48.0
1,5001710,-41.0
2,5001711,-1.5
3,5001712,-9.0
4,5001713,-10.5


In [151]:
bureau_bal_mean = bureau_bal_mean.rename(columns = {'BUR_BAL_MEAN_SK_ID_BUREAU' : 'SK_ID_BUREAU'})

In [152]:
final_dataset = final_dataset.merge(bureau_bal_mean,  on = 'SK_ID_BUREAU', how = 'left')
final_dataset.drop('SK_ID_BUREAU', axis = 1, inplace = True) # we don't need this internal ID anymore

In [153]:
def extract_mean_bur(x):
    y = x.groupby('SK_ID_CURR', as_index=False).mean().add_prefix('BUR_MEAN_')
    return y

In [154]:
final_data_mean_values = extract_mean_bur(final_dataset)
final_data_mean_values = final_data_mean_values.rename(columns = {'BUR_MEAN_SK_ID_CURR' : 'SK_ID_CURR'})

C:\Users\Admin\AppData\Local\Temp\ipykernel_4528\1038745899.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  y = x.groupby('SK_ID_CURR', as_index=False).mean().add_prefix('BUR_MEAN_')


In [156]:
final_data_mean_values.head()

,SK_ID_CURR,BUR_MEAN_DAYS_CREDIT,BUR_MEAN_CREDIT_DAY_OVERDUE,BUR_MEAN_AMT_CREDIT_MAX_OVERDUE,BUR_MEAN_CNT_CREDIT_PROLONG,BUR_MEAN_AMT_CREDIT_SUM,BUR_MEAN_AMT_CREDIT_SUM_DEBT,BUR_MEAN_AMT_CREDIT_SUM_LIMIT,BUR_MEAN_AMT_CREDIT_SUM_OVERDUE,BUR_MEAN_AMT_ANNUITY,BUR_MEAN_DEBT_CREDIT_RATIO,BUR_MEAN_PAST_LOAN_COUNT,BUR_MEAN_CREDIT_TYPE_COUNT,BUR_MEAN_ACTIVE_LOANS_PERCENTAGE,BUR_MEAN_BUR_BAL_MEAN_MONTHS_BALANCE
0,0,-63.000000,0.0,NaN,0.0,85513.500000,77566.500000,NaN,0.0,NaN,90.706731,1.0,1.0,100.000000,NaN
1,1,-2348.000000,0.0,11666.385,0.0,28350.000000,0.000000,0.0,0.0,NaN,NaN,1.0,1.0,100.000000,-50.5
2,2,-810.333333,0.0,3771.990,0.0,42390.180000,16069.500000,0.0,0.0,NaN,37.908544,6.0,1.0,33.333333,NaN
3,3,-1131.428571,0.0,0.000,0.0,69136.071429,28395.642857,0.0,0.0,NaN,41.072109,7.0,2.0,42.857143,NaN
4,4,-773.333333,0.0,NaN,0.0,470424.000000,209084.250000,0.0,0.0,NaN,44.445915,6.0,2.0,50.000000,NaN


In [157]:
final_data_mean_values.to_csv('bureau_balance_merge_bureau.csv', index = False)